In [35]:
!pip install transformers
!pip install gensim
!pip install datasets
!pip install evaluate
!pip install torch
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from datasets import load_dataset
import numpy as np
import transformers
from transformers import DistilBertModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import EncoderDecoderModel, BertConfig, EncoderDecoderConfig, EncoderDecoderModel
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchtext
from torchtext.vocab import Vocab
import torch.nn.utils.rnn as rnn_utils
import gensim
from gensim.models import Word2Vec, KeyedVectors
import evaluate
from collections import Counter

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
collator = DataCollatorWithPadding(tokenizer=tokenizer)

def bert_tokenizer(data):
  token = tokenizer(data['text'], padding='max_length', truncation=True)
  return token

ds = load_dataset('imdb')
ds = ds.map(bert_tokenizer, batched=True)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [54]:
train_data = ds["train"].shuffle(seed=0).select(range(1000))
eval_data = ds["test"].shuffle(seed=0).select(range(1000))
ft_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
lp_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
ft_params = TrainingArguments(output_dir='finetuning_training', evaluation_strategy='epoch')
lp_params = TrainingArguments(output_dir='linearprobing_training', evaluation_strategy='epoch')
def compute_metrics(pred):
  metric = evaluate.load('glue', 'mrpc')
  logits, labels = pred
  predictions = np.argmax(logits, axis=-1)
  acc = metric.compute(predictions=predictions, references=labels)
  return acc

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
ft_trainer = Trainer(model=ft_model,
                  args=ft_params,
                  train_dataset=train_data,
                  eval_dataset=eval_data,
                  data_collator=collator,
                  compute_metrics=compute_metrics)
ft_trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.322396,0.866000,0.849776
2,No log,0.380734,0.894000,0.895050
3,No log,0.415063,0.903000,0.901322


TrainOutput(global_step=375, training_loss=0.29174704996744794, metrics={'train_runtime': 403.5428, 'train_samples_per_second': 7.434, 'train_steps_per_second': 0.929, 'total_flos': 789333166080000.0, 'train_loss': 0.29174704996744794, 'epoch': 3.0})

In [ ]:
for param in lp_model.base_model.parameters():
  param.requires_grad = False

lp_trainer = Trainer(model=lp_model,
                  args=lp_params,
                  train_dataset=train_data,
                  eval_dataset=eval_data,
                  data_collator=collator,
                  compute_metrics=compute_metrics)
lp_trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.684604,0.574000,0.607735
2,No log,0.681375,0.592000,0.601562
3,No log,0.680574,0.595000,0.624652


TrainOutput(global_step=375, training_loss=0.6853706868489583, metrics={'train_runtime': 215.1182, 'train_samples_per_second': 13.946, 'train_steps_per_second': 1.743, 'total_flos': 789333166080000.0, 'train_loss': 0.6853706868489583, 'epoch': 3.0})

In [ ]:
ft_predictions = ft_trainer.predict(eval_data)
print(ft_predictions.metrics)

{'test_loss': 0.41506293416023254, 'test_accuracy': 0.903, 'test_f1': 0.9013224821973549, 'test_runtime': 35.2321, 'test_samples_per_second': 28.383, 'test_steps_per_second': 3.548}


In [ ]:
lp_predictions = lp_trainer.predict(eval_data)
print(lp_predictions.metrics)

{'test_loss': 0.6805742383003235, 'test_accuracy': 0.595, 'test_f1': 0.6246524559777571, 'test_runtime': 35.2857, 'test_samples_per_second': 28.34, 'test_steps_per_second': 3.543}


In [64]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
corpus = []
for data in ds['train']['text']:
  tokens = tokenizer(data)
  corpus.append(tokens)

counts = []
for data in corpus:
  counts.append(Counter(data, min_freq=1))
vocabulary = Vocab(counts)
itos = vocabulary.get_itos

In [51]:
class Transformer(nn.Module):
  def __init__(self, 
               d_model, 
               dff, 
               embed_size, 
               num_layers, 
               num_heads,
               hidden_size, 
               dropout_rate,
               target_vocab_size,
               embeddings=None):
    super(Transformer, self).__init__()
    d_model
    self.d_model = d_model
    self.dff = dff
    self.embed_size = embed_size
    self.num_heads = num_heads
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.dropout = nn.Dropout(dropout_rate)
    self.target_vocab_size = target_vocab_size
    self.embeddings = nn.Embedding.from_pretrained(embeddings) if embeddings is not None else None

    self.encoder_layer = nn.TransformerEncoderLayer(nhead=num_heads, d_model=d_model)
    self.decoder_layer = nn.TransformerDecoderLayer(nhead=num_heads, d_model=d_model)
    self.encoder = nn.TransformerEncoder(self.encoder_layer, 6)
    self.decoder = nn.TransformerDecoder(self.decoder_layer, 6)

  def forward(self, x):
    if self.embeddings is not None:
      x = self.embeddings(x)
      x *= self.d_model ** 0.5  # scaling the embeddings of the model
    x = self.encoder(x)
    x = self.decoder(x, x)
    x = x[:, 0 , :]
    return x

In [ ]:
# getting glove model 
glove = torchtext.vocab.GloVe(name='6B', dim=300, unk_init=torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.34MB/s]                           
100%|█████████▉| 399999/400000 [00:47<00:00, 8385.46it/s]


In [ ]:
torch.save(glove, 'glove.bin')

In [ ]:
glove = torch.load('glove.bin')

In [ ]:
embeddings = []
for key in ds['train']['text']:
  if key in glove.stoi:
    embeddings.append(glove[key])
  else:
    embeddings.append(np.random.randn(300))

embeddings = torch.tensor(embeddings)

<ipython-input-9-a132caf90684>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  embeddings = torch.tensor(embeddings)


In [66]:
embedding_dims = 300
d_model = embedding_dims
dff = 2 * d_model
num_layers = 2
num_heads = 2
hidden_size = 128
dropout_rate = 0.1
target_vocab_size = 2

transformer = Transformer(d_model=d_model,
                          dff=dff, 
                          embed_size=embedding_dims, 
                          num_layers=num_layers,
                          num_heads=num_heads, 
                          hidden_size=hidden_size,
                          dropout_rate=dropout_rate, 
                          target_vocab_size=target_vocab_size, 
                          embeddings=embeddings)
num_classes = 2
model = nn.Sequential(transformer, nn.Linear(d_model, num_classes))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

model_params = TrainingArguments(output_dir='model_training', evaluation_strategy='epoch')
model_trainer = Trainer(model=model,
                        args=model_params,
                        train_dataset=train_data,
                        eval_dataset=eval_data,
                        compute_metrics=compute_metrics)
model_trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TypeError: ignored